In [1]:
import sys
import os
sys.path.append(os.path.abspath("../"))
sys.path.append(os.path.abspath("../.."))
from fashion_class.Coordinate import Coordinate
from fashion_class.ImageStruct import ImageStruct
import pandas as pd
import json

In [2]:
# データセットを初期化
annotation_file, tensor_file = \
      '../data/anotation_new.csv', '../data/resnet50_tensor.pt'
dataset = ImageStruct(annotation_file, tensor_file, init_item_length = 300)

【================================================================================================    】

In [8]:
with open('D:\M1/fashion/lda_model/data/fashion_clip.txt', mode="r", encoding="utf-8") as f:
    data = f.read().splitlines()
item_info = {}
for i in data:
    i = i.strip()
    item_id, category, color = i.split(' ')
    item_info[int(item_id)] = f'{category} {color}'


In [9]:
tops = dataset.tops
bottoms = dataset.bottoms
shoes = dataset.shoes

In [10]:
from fashion_class.FashionItem import FashionItem
import numpy as np
import tomotopy as tp
from lda_model.util.ignore_items import is_ignore_items
from lda_model.util.is_stopword import is_stopword
from lda_model.preProcessing.preprocessing import preprocessing

from lda_model.util.parse_sentence import parse_sentence

# モデルを読み込む
# D:\M1\fashion\optimization\experiments\lda-fashion-clip-T-100-M-10000-B-1000.bin
mdl = tp.LDAModel.load('lda-fashion-clip-T-100-M-10000-B-1000.bin')
def get_lda_base_compatibility(t: FashionItem, b: FashionItem, s: FashionItem, mdl):
    doc = []
    for item in [t, b, s]:
        item = item.item_info
        if ("itemId" not in item or item["itemId"] == None):
            continue
        item_id = item["itemId"]
        if item_id not in item_info:
            continue
        doc.append(item_info[item_id])
    print(doc)
    if len(doc) == 0:
        return 0
    inf_doc = mdl.make_doc(doc)
    log_prob = mdl.infer(inf_doc, iter=500)[1]
    return log_prob


In [11]:
# # c = 0
# length = len(tops) * len(bottoms) * len(shoes)
# print(length)

# for t in tops:
#     for b in bottoms:
#         for s in shoes:
#             c += 1
#             coordinate = Coordinate(t, b, s)
#             compatibility = coordinate.get_compatibility()
#             com.append(compatibility)
#             lda_base.append(get_lda_base_compatibility(t, b, s, mdl))
#             if (c % 1000 == 0):
#                 print(f"\r {c * 100 // length}%終了", end='')



In [12]:
with open('D:/M1/fashion/optimization/data/shoes_count.json', 'r', encoding='shift-jis') as f:
    new_compatibility = json.load(f)

In [26]:
count = {}
lda_base = []
com = []
KEY = "category x color"
with open('D:\M1/fashion/lda_model/data/filepath/train.txt', 'r', encoding="utf-8") as f:
    filepaths = f.read().splitlines()
filepaths = filepaths[:50000]
file_count = len(filepaths)
# output_file = "D:\M1/fashion/lda_model/data/fashion_clip/train.txt"
count = 0
for fp in filepaths:
    count += 1
    try:
        json_dict = pd.read_json(fp, encoding='shift-jis')
    except Exception as e:
        continue
    doc = []
    flags = [True, True, True]
    items = [None, None, None]
    for item in json_dict["items"]:
        if ("itemId" not in item or item["itemId"] == None):
            continue
        if (KEY not in item or len(item[KEY]) == None):
            continue
        item_id = item["itemId"]
        category = item[KEY].strip()
        if item_id not in item_info:
            continue
        c = FashionItem.get_category(None, category.split(' × ')[0])
        q, w = item_info[item_id].split(' ')
        doc.append(q)
        doc.append(w)
        if c == 'tops' and flags[0]:
            flags[0] = False
            items[0] = category
        elif c == 'bottoms' and flags[1]:
            flags[1] = False
            items[1] = category
        elif c == 'shoes' and flags[2]:
            flags[2] = False
            items[2] = category
        

    if len(doc) == 0 or None in items:
        continue
    inf_doc = mdl.make_doc(doc)
    log_prob = mdl.infer(inf_doc, iter=500)[1]
    lda_base.append(log_prob)
    com.append(new_compatibility[items[0]][items[1]][items[2]])

    if (count % 10 == 0):
        print(f"\r {count * 100 // file_count}%終了", end='')

        

 100%終了

In [27]:
with open('compatibility.txt', 'w') as f:
    for (c1, c2) in zip(com, lda_base):
        # print(c1, c2)
        f.write(f'{float(c1):.5} {float(c2):.5}\n')

'ニット × 黒（ブラック）'